This Notebook is used to create and train a neural network model for the purpose of predicting the worm movement.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd

from wtracker.utils.path_utils import Files
from wtracker.utils.config_base import print_initialization
from wtracker.neural.config import DatasetConfig, TrainConfig, IOConfig, LOSSES, OPTIMIZERS
from wtracker.neural.dataset import NumpyDataset
from wtracker.neural.mlp import MlpBlock, RMLP, WormPredictor
from wtracker.neural.training import MLPTrainer
from wtracker.neural.train_results import FitResult
from wtracker.utils.gui_utils import UserPrompt


pd.options.display.max_columns = 30

torch.set_printoptions(sci_mode=False)

ModuleNotFoundError: No module named 'tkfilebrowser'

### Configure the Model, Dataset and Training Parameters

In [ ]:
################################ User Input ################################

# path to the log file, used to train the network
# if None, a file dialog will open to select the file
log_path = None 

io_config = IOConfig(
    input_frames=[0, -3, -15, -18, -30],  # list
    pred_frames=[12],  # list
)

############################################################################

if log_path is None:
    log_path = UserPrompt.open_file("Please select the log file to use for training")

dataset_config = DatasetConfig.from_io_config(io_config, log_path) # create a dataset config object from the io_config and log_path

In [ ]:
# create a neural network model based on the dataset_config. Will be ignored if a model path is given by TrainConfig
block_in_dim = 40
block_dims = [10, 4, 10, 40]
block_activations = ["relu"] * (len(block_dims))
in_dim = 4 * len(dataset_config.input_frames)
out_dim = 2 * len(dataset_config.pred_frames)

model = RMLP(
    in_dim=in_dim,
    block_in_dim=block_in_dim,
    block_dims=block_dims,
    block_nonlins=block_activations,
    n_blocks=4,
    out_dim=out_dim,
)

In [ ]:
# we wrap the model in a WormPredictor object, which will hold the io_config for future use and distinguish it from general Neural Network models.
model = WormPredictor(model, io_config)

In [ ]:
print(model) # print the model layers

In [ ]:
################################ User Input ################################

train_config = TrainConfig(
    seed=42,  # int
    dataset=dataset_config,  # Dataset
    model=model,  # Union[nn.Module, str]
    loss_fn="mse",  # nn.Module
    optimizer="adam",  # Union[Optimizer, str]
    device=f"cpu",  # str
    log=True,  # bool
    num_epochs=100,  # int
    checkpoints="RMLP4",  # str
    early_stopping=15,  # int
    print_every=5,  # int
    learning_rate=0.001,  # float
    weight_decay=1e-05,  # float
    batch_size=256,  # int
    shuffle=True,  # bool
    num_workers=0,  # int
    train_test_split=0.8,  # float
)

############################################################################

### Run The Training Process

In [ ]:
# create a dataset according to the dataset_config
dataset = NumpyDataset.create_from_config(dataset_config, save_path=None)

In [ ]:
# Split the dataset
ds_train, ds_test = random_split(dataset, [train_config.train_test_split, 1 - train_config.train_test_split])

In [ ]:
# Create the dataloaders
dl_train = DataLoader(ds_train, batch_size=train_config.batch_size, shuffle=train_config.shuffle)
dl_test = DataLoader(ds_test, batch_size=train_config.batch_size, shuffle=train_config.shuffle)

In [ ]:
# initialize the loss object
loss_fn = LOSSES[train_config.loss_fn]()

In [ ]:
# initialize the optimizer object
lr = train_config.learning_rate
weight_decay = train_config.weight_decay
optimizer = OPTIMIZERS[train_config.optimizer](model.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
# create the trainer object
device = torch.device(train_config.device)
trainer = MLPTrainer(model, loss_fn, optimizer, device=device, log=train_config.log)

In [ ]:
# train the model
epochs = train_config.num_epochs
checkpoints = train_config.checkpoints
early_stopping = train_config.early_stopping
print_every = train_config.print_every

trainer.fit(
    dl_train=dl_train,
    dl_test=dl_test,
    num_epochs=epochs,
    checkpoints=checkpoints,
    print_every=print_every,
    early_stopping=early_stopping,
)

In [ ]:
# save the training configuration if logging is enabled
if train_config.log:
    train_config.save_pickle(trainer.logger.log_dir + "/train_config.pkl")